# Step 1 :-  Prepare Training Data
The first step in building any tokenizer is to have a corpus of text to train it on.The Tokenizer learns *merge rules* based on the frequency of the character pairs in the data.

1. i : 1
2. s: 2
3. is : 3

Even thogh "i" and "s" are seperate tokens, we create a new token "is" by merging them as thy frequently appear together (is, this, miss, dismiss, list, fist, twist etc) reducing computations by 2x where we merge those two tokens "i" and "s". This is how we will iteratively merge most frequent pairs. The  new tokens can also be merged further.

#### training corpus:
#### Category: Conversational
---
User: Hey, how's it going?
Bot: I'm doing well! How about you?
User: Pretty good, just learning about AI.
Bot: That sounds fascinating! What aspects interest you the most?

#### Category: Technical
---
Sentence: Tokenization breaks text into smaller units called tokens.
Sentence: Embeddings represent words as numerical vectors in a multidimensional space.
Sentence: Sampling methods influence how a language model selects its next word.

#### Category: Storytelling
---
Once upon a time, in a distant galaxy, a lone explorer embarked on a mission to uncover the secrets of an ancient civilization. As they decoded mysterious symbols, the truth about their origins slowly unfolded…

#### Category: Formal Writing
---
Dear Hiring Manager,  
I am writing to express my interest in the Data Science position at your company. With a background in machine learning and AI research, I believe my experience aligns well with the role. I look forward to the opportunity to contribute to your team.


# Step - 2: initalize vocabulary and pre-tokenize

The BPE algorithm starts with a base vocab consisting of all unique chracters present in the training data.
 
We also need to pre-tokenize the corpus. This usually involves splitting the text into words (or words like units), and then representing each word as a sequence of its individual characters. We often add a end-of-word token(</w>)
to mark word boundaries.

In [1]:
corpus = [ "Once upon a time, in a distant galaxy, a lone explorer embarked on a mission to uncover the secrets of an ancient civilization. As they decoded mysterious symbols, the truth about their origins slowly unfolded…",
              "In a world where technology and magic coexisted, a young sorcerer discovered a hidden power within themselves. With the help of an unlikely ally, they set out to master their abilities and confront an impending darkness that threatened their realm.",
                "In a post-apocalyptic wasteland, a group of survivors banded together to rebuild society. As they faced external threats and internal conflicts, they learned the true meaning of community and resilience.",
                "In a bustling metropolis, a detective with a troubled past found themselves entangled in a web of crime and corruption. As they navigated the shadows of the city, they uncovered a conspiracy that reached the highest echelons of power.",
                "In a small town, a mysterious stranger arrived, bringing with them tales of adventure and intrigue. As the townsfolk became embroiled in their stories, they discovered hidden truths about themselves and their community.",
                "In a fantasy realm, a young hero set out on a quest to retrieve a stolen artifact that held the key to restoring balance to their world. Along the way, they encountered mythical creatures, treacherous landscapes, and unexpected allies.",
                "In a dystopian future, a group of rebels fought against an oppressive regime. As they risked everything for freedom, they discovered the power of hope and the strength of the human spirit.",
                "In a historical setting."
    ]

In [2]:
unique_char = set()  # set does not allow duplicates
# Iterate through each sentence in the corpus
for sentence in corpus:
    # Iterate through each character in the sentence
    for char in sentence:
        # Check if the character is not already in the set
        if char not in unique_char:
            # If not, add it to the set
            print(char)
            # Add the character to the set
        unique_char.add(char)

O
n
c
e
 
u
p
o
a
t
i
m
,
d
s
g
l
x
y
r
b
k
v
h
f
z
.
A
w
…
I
W
-
q


In [3]:
vocab = list(unique_char)  # Convert the set to a list
vocab.sort()  # sort the list

# add end of word char
vocab.append('</w>')

print(vocab)
print("len of vocabulary: ", len(vocab))

[' ', ',', '-', '.', 'A', 'I', 'O', 'W', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '…', '</w>']
len of vocabulary:  35


In [7]:
end_of_word = vocab[34]  # index of end of word char

In [8]:
end_of_word

'</w>'

### Pre-tokenization

In [ ]:
word_splits = {}
for sentence in corpus:
    # Split the sentence into words
   words = sentence.split(" ")
   for word in words:
      if word:
         char_list = list(word) + [end_of_word]

         word_tuple = tuple(char_list)
         if word_tuple not in word_splits:
            word_splits[word_tuple] = 1
         word_splits[word_tuple] += 1  # count frequency of word
print("word splits: ", word_splits)
         

word splits:  {('O', 'n', 'c', 'e', '</w>'): 2, ('u', 'p', 'o', 'n', '</w>'): 2, ('a', '</w>'): 24, ('t', 'i', 'm', 'e', ',', '</w>'): 2, ('i', 'n', '</w>'): 4, ('d', 'i', 's', 't', 'a', 'n', 't', '</w>'): 2, ('g', 'a', 'l', 'a', 'x', 'y', ',', '</w>'): 2, ('l', 'o', 'n', 'e', '</w>'): 2, ('e', 'x', 'p', 'l', 'o', 'r', 'e', 'r', '</w>'): 2, ('e', 'm', 'b', 'a', 'r', 'k', 'e', 'd', '</w>'): 2, ('o', 'n', '</w>'): 3, ('m', 'i', 's', 's', 'i', 'o', 'n', '</w>'): 2, ('t', 'o', '</w>'): 7, ('u', 'n', 'c', 'o', 'v', 'e', 'r', '</w>'): 2, ('t', 'h', 'e', '</w>'): 14, ('s', 'e', 'c', 'r', 'e', 't', 's', '</w>'): 2, ('o', 'f', '</w>'): 12, ('a', 'n', '</w>'): 5, ('a', 'n', 'c', 'i', 'e', 'n', 't', '</w>'): 2, ('c', 'i', 'v', 'i', 'l', 'i', 'z', 'a', 't', 'i', 'o', 'n', '.', '</w>'): 2, ('A', 's', '</w>'): 6, ('t', 'h', 'e', 'y', '</w>'): 11, ('d', 'e', 'c', 'o', 'd', 'e', 'd', '</w>'): 2, ('m', 'y', 's', 't', 'e', 'r', 'i', 'o', 'u', 's', '</w>'): 3, ('s', 'y', 'm', 'b', 'o', 'l', 's', ',', '</

In [37]:
import collections

def get_pair_stats(splits): # get the frequency of pairs of characters
    pair_counts = collections.defaultdict(int)
    #print("splits: ", splits)
    for word, freq in splits.items():
        #print("word: ", word)       # LOGGING
        #print("freq: ", freq)  
        word = list(word) 
        #print(word)    # LOGGING
        for i in range(len(word), -1):
            #print("i: ", i)           # LOGGING
            
            pair = (word[i], word[i+1])
            pair_counts[pair] += freq
    return pair_counts

In [38]:
get_pair_stats(word_splits)

defaultdict(int, {})

In [39]:
pair_counts = get_pair_stats(word_splits)

In [33]:
print("pair counts: ", pair_counts)

pair counts:  defaultdict(<class 'int'>, {})


In [40]:
def merge_pair(pairs_to_merge, splits):
    # Merge the most frequent pair in the splits
    new_splits = {}
    for word, freq in splits.items():
        new_word = []
        i = 0
        while i < len(word):
            pair = (word[i], word[i+1])
            if pair == pairs_to_merge:
                new_word.append(word[i] + word[i+1])
                i += 2
            else:
                new_word.append(word[i])
                i += 1
        new_word = tuple(new_word)
        if new_word not in new_splits:
            new_splits[new_word] = 0
        new_splits[new_word] += freq
    return new_splits
    

In [ ]:
# BPE
